**IDEE:**
- ein Interface (UI) als frontend, während der nachfolgende Code im backend das matching durchführt
- user sollte die zwei Input files über entsprechende UI elemente hochladen können (master und using file)
- user sollte zudem über entsprechende text felder die relevanten Spaltennamen spezifizieren können (M_ID, M_NAME, M_YEAR, M_QTR sowie P_xx im Code unten)
- es sollte eine Fortschrittsanzeige angezeigt werden
- abschließend sollte der user die Ergebnisse über einen download button exportieren können

In [2]:
import re
from pathlib import Path

import pandas as pd
from thefuzz import fuzz, process
from tqdm import tqdm

In [3]:
def normalize_company_names(x: str):
    '''Clean company names and prepare for fuzzy matching'''
    # lowercasing
    x = str(x).lower().strip()
    # remove special characters
    x = re.sub(r'[^a-z0-9 ]', '', x)
    # remover certain trailing artifacts
    x = re.sub(r' (q[1-4].*?|old|adr)$', '', x)     # SA / Compustat
    x = re.sub(r' (cl a|cl b|redh)$', '', x)        # ExecuComp
    # remove legal forms
    x = re.sub(r' (public limited company|public limited|limited|unlimited|partnership|incorporation|incorporated|corporation|plc|pbc|ltd|inc|corp|llc|lp)$', '', x)
    # normalize other frequent error cases
    x = re.sub(r' co$', ' company', x)
    x = re.sub(r'^the ', '', x)
    x = re.sub(r' and ', ' ', x)
    # remove of whitespaces
    x = x.replace(' ', '')
    return x


def normalize_person_names(x):
    '''Clean company names and prepare for fuzzy matching'''
    # remove special characters
    x = re.sub(r'[^A-Za-z0-9 ]', '', x)
    # remove title
    x = re.sub(r' (Economics|PharmD|CISA|MPPM|Hons|Hon|BBA|MBA|JD|MIM|PhD|Hon|FCPA|CFA|CPA|FCA|CMA|MAI|BSc|BSC|MSc|MSC|ESQ|MS|BA|CA|BE|AM|PE|AO|MD)', '', x)
    # lowercasing
    x = str(x).lower().strip()    
    # sort names alphabetically
    x = x.split()
    x.sort()
    x = ''.join(x)
    return x


def fuzzy_match(query:pd.DataFrame, q_name_norm:str,
                db:pd.DataFrame, db_name_norm:str, db_name_id:str, db_name:str,
                q_fy:str=None, q_qtr:str=None, db_fy:str=None, db_qtr:str=None):
    '''Fuzzy match nearest neighbours for query in (filtered) database based on Levenshtein Distance
    
    :param pd.DataFrame query:
        DataFrame from which to query
    :param str q_name_norm:
        Column name of normalized string column in query
        
    :param pd.DataFrame db:
        DataFrame in which to search for match
    :param str db_name_norm:
        Column name of normalized string column in database
    :param str db_name_id:
        Column name of id column in database
    :param str db_name:
        Column name of string column in database
        
    :param str q_fy:
        Column name of fiscal year column in query (for filtering)
    :param str q_qtr:
        Column name of fiscal quarter column in query (for filtering)
    :param str db_fy:
        Column name of fiscal year column in database (for filtering)
    :param str db_qtr:
        Column name of fiscal quarter column in database (for filtering)
        
    :return pd.Series neihbours:
        Series of tuples of the form (nearest neighbour:str, similarity score:int, gvkey:int) 
        
    '''
    
    def retrieve_nn(x:pd.DataFrame, db:pd.DataFrame):
        ''' Fuzzy match nearest neighbour for query string '''
        
        # limit fuzzy match to relevant time interval
        if q_fy and q_qtr and db_fy and db_qtr:
            db = db[(db[db_fy]==x[q_fy]) & (db[db_qtr]==x[q_qtr])]
        elif q_fy and db_fy:
            db = db[(db[db_fy]==x[q_fy])]
        
        # retrieve nearest neighbour
        try:
            nn, score, _ = process.extractOne(x[q_name_norm], db[db_name_norm], scorer=fuzz.ratio)
            id = db[db[db_name_norm]==nn][db_name_id]
            id = id.item() if len(id)==1 else None
            name = db[db[db_name_norm]==nn][db_name]
            name = name.item() if len(name)==1 else None
            return (nn, score, id, name)
        except:
            return (None, None, None, None)
     
    tqdm.pandas()
    return query.progress_apply(lambda x: retrieve_nn(x, db), axis=1)

In [12]:
# parameters

F_EXTENSION = '.csv'

PATH_M = Path('input', f'master{F_EXTENSION}')
M_ID = 'secid'
M_NAME = 'cname'
M_YEAR = None
M_QTR = None

PATH_U  = Path('input', f'using{F_EXTENSION}')
U_ID = 'gvkey'
U_NAME = 'coname'
U_YEAR = None
U_QTR = None

OUT = 'matches'

In [13]:
if F_EXTENSION == '.csv':
    master = pd.read_csv(PATH_M)
    using = pd.read_csv(PATH_U)
elif F_EXTENSION == '.dta':
    master = pd.read_stata(PATH_M)
    using = pd.read_stata(PATH_U)
    
M_NAME_NORM = f'{M_NAME}_norm'
U_NAME_NORM = f'{U_NAME}_norm'

master = master.assign(**{M_NAME_NORM: master[M_NAME].map(lambda x: normalize_company_names(x))})
using = using.assign(**{U_NAME_NORM: using[U_NAME].map(lambda x: normalize_company_names(x))})

In [14]:
master['nn_match'], master['nn_score'], master[f'nn_{U_ID}'], master[f'nn_{U_NAME}'] = \
    zip(*fuzzy_match(master, M_NAME_NORM,
                     using, U_NAME_NORM, U_ID, U_NAME,
                     q_fy=M_YEAR, q_qtr=M_QTR, db_fy=U_YEAR, db_qtr=U_QTR))

100%|██████████| 1071/1071 [00:15<00:00, 69.82it/s]


In [16]:
if F_EXTENSION == '.csv':
    master.to_csv(Path('output', OUT).with_suffix('.csv'))
elif F_EXTENSION == '.dta':
    master.to_stata(Path('output', OUT).with_suffix('.dta'), version=118)